In [1]:
#!pip install torchvision
# Convolutional neural nets

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

In [3]:
trainset = datasets.MNIST('Datasets', download=True, train=True, transform=transforms.ToTensor())
testset = datasets.MNIST('Datasets', download=True, train=False, transform=transforms.ToTensor())

trainloader = DataLoader(trainset, shuffle=True)
testloader = DataLoader(testset, shuffle=True)

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.sigm1 = nn.Sigmoid()
        self.avgp1 = nn.AvgPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.sigm2 = nn.Sigmoid()
        self.avgp2 = nn.AvgPool2d(2, stride=2)
        
        self.fc1 = nn.Linear(256, 120)
        self.sigm3 = nn.Sigmoid()
        
        self.fc2 = nn.Linear(120, 84)
        self.sigm4 = nn.Sigmoid()
        
        self.fc3 = nn.Linear(84, 10)
        self.sigm5 = nn.Sigmoid()
        
    def forward(self, x):
        y = self.conv1(x)
        y = self.sigm1(y)
        y = self.avgp1(y)
        
        y = self.conv2(y)
        y = self.sigm2(y)
        y = self.avgp2(y)
        
        y = y.reshape(-1) # convert 5x5x16 to 400x1
        
        y = self.fc1(y)
        y = self.sigm3(y)
        
        y = self.fc2(y)
        y = self.sigm4(y)
        
        y = self.fc3(y)
        y = self.sigm5(y)
        return y

In [9]:
from tqdm import tqdm

model = LeNet()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

for train_x, train_y in tqdm(trainloader):
    optimizer.zero_grad()
        
    predict_y = model(train_x.float())
    predict_y = predict_y.reshape(1, -1)
    
    loss = criterion(predict_y, train_y)
    loss.backward()
    optimizer.step()
print("End of train")

 
print("Begin test set estimation")
correct = 0 
total_test = len(testset)
with torch.no_grad():
    for image, label in tqdm(testloader):
        y = model(image)
        y = y.reshape(1, -1)
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == label).float())
print(f"\nTest set accuracy: {(correct/total_test):.4}")    

100%|███████████████████████████████████████████████████████████████████████████████████| 60000/60000 [03:50<00:00, 260.38it/s]


End of train
Begin test set estimation


100%|███████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 883.70it/s]


Test set accuracy: 0.1135
